## Support Vector Machine for tagging GOV.UK

### Load requirements and data

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.svm import LinearSVC

In [ ]:
# Double check we are in the right environment!
!which python3
!python3 --version

In [3]:
# Becuase the file is large it uses the low_memory option 
# to read the file in chunks. The result of this is that it
# it infers different column types in the different chunks,
# solve this either by reading the file in a oner with low_memory=False,
# or be explicit about the column types by setting them with dtype = {'column':type}

labelled_level2 = pd.read_csv(
    '../../../../data/labelled_level2.csv',
    low_memory=False
)

In [4]:
labelled_level2.shape

(173560, 21)

In [5]:
# Collapse down World

labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

In [6]:
# There are are 210 unique taxons remaining in level2taxon

labelled_level2['level2taxon'].nunique()

210

In [7]:
# Drop cases where there are multiple taxons applied to a content item.
# This is the simplest solution to the multiple class problem

print('Before:', labelled_level2.shape)
labelled_level2.drop_duplicates(subset=['content_id'], inplace=True, keep='first')
print('After: ', labelled_level2.shape)

Before: (173560, 21)


In [8]:
comment_pipeline = Pipeline([
                   ('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer(sublinear_tf=True, use_idf=True)),
                   ('anova', SelectPercentile())
])

In [9]:
text = comment_pipeline.fit_transform(labelled_level2['combined_text'],labelled_level2['level2taxon'])

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [10]:
text.shape

(173560, 16854)

In [ ]:
metadata = labelled_level2.filter(
    ['document_type', 'primary_publishing organisation', 
     'publishing_app', 
     'document_type_gp', 'locale'], 
    axis=1)

In [ ]:
pd.concat([text,m])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    text, labelled_level2['level2taxon'], test_size = 0.2, random_state=1337)

In [13]:
svc_clf = Pipeline([
                   ('svc', LinearSVC(C=10.0, dual=False, loss="squared_hinge", penalty="l2", tol=0.01))
                   ])

In [14]:
svc_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('svc', LinearSVC(C=10.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.01,
     verbose=0))])

In [15]:
""" Predict the test dataset using SVM"""
predicted_svm = svc_clf.predict(X_test)
print('SVM correct prediction: {:4.2f}'.format(np.mean(predicted_svm == y_test)))

SVM correct prediction: 0.45


In [21]:
print(classification_report(y_test, predicted_svm))


/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


str

## Metadata classifier

In [ ]:
labelled_level2.columns

In [ ]:
metadata = labelled_level2.filter(['document_type', 'primary_publishing organisation', 'publishing_app', 'document_type_gp', 'locale'], axis=1)

In [ ]:
metadata.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    metadata, labelled_level2['level2taxon'], test_size = 0.2, random_state=1337)

In [ ]:
ada_clf = AdaBoostClassifier()

ada_clf.fit(X_train, y_train)

In [ ]:
predicted_ada = ada_clf.predict(X_test)

In [ ]:
print('Adaboost correct prediction: {:4.2f}'.format(np.mean(predicted_ada == y_test)))

In [ ]:
print(classification_report(y_test, predicted_ada))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfo_clf = RandomForestClassifier()
rfo_clf.fit(X_train, y_train)

In [ ]:
predicted_rfo = rfo_clf.predict(X_test)
print('RandomForest correct prediction: {:4.2f}'.format(np.mean(predicted_rfo == y_test)))

In [ ]:
print(classification_report(y_test, predicted_ada))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
predicted_rfo = rfo_clf.predict_proba(X_test)
predicted_ada = ada_clf.predict_proba(X_test)

In [ ]:
blended = pd.DataFrame(np.c_[
    predicted_rfo, predicted_ada
])

In [ ]:
blended_model = LogisticRegression(random_state=1337)
blended_model.fit(blended, y_test)

In [ ]:
blended_pred = blended_model.predict(blended)
class_report = classification_report(y_test, blended_pred)
print(class_report)
blended_model.get_params()

In [ ]:
# Double check this with CV

scores = cross_val_score(blended_model, X_test, y_test,
                        scoring='accuracy',cv=10)

print(round(scores.mean(),3))
print(round(scores.std(),3))
#72